# Colored Water Pots Problem - First Project - Knowledge Representation

## Imports 

In [1]:
import os
import copy
import sys
import time

## Classes 

### Node Class

**Node class constructor:**
* **info** - type: list(dictionary) - contains informations about the water pots 

    info =[ { "id" : number, "capacity" : number, "quantity" : number, "color" : string }, ... ]
    
* **cost** - type: number - is the cost of the path to the current node
* **heuristic**  - type: number - is the cost of the path from the current node to the destination node
* **estimated_cost** - type: number - is cost + heuristic
* **parent** : type: Node - is the current node's parent in the path
* **path_info** - type: dictionary - contains informations about the combanation of the pots

    path_info = { "first_pot" : potInfo, "second_pot" : potInfo, "color" : string, "poured_liters" : number }

In [2]:
class Node:
    
    # Class Constructor
    # Node (info, parent, cost, h, pathInfo)
    graph = None
    
    def __init__(self, info: list, parent,  cost = 0, heuristic = 0, path_info = {}):
        # type: list(dictionary) - contains informations about the water pots
        self.info = info
        
        # type: Node - is the current node's parent in the path
        self.parent = parent
        
        # type: number - is the cost of the path to the current node
        self.cost = cost
        
        # type: number - is the cost of the path from the current node to the destination node
        self.heuristic = heuristic
        
        # type: number - is cost + heuristic
        self.estimated_cost = self.cost + self.heuristic
        
        # type: type: dictionary- contains informations about the combanation of the pots
        self.path_info = path_info
    
    def __str__(self):
        result_string = ""
        
        # { "id" : number, "capacity" : number, "quantity" : number, "color" : string }
        for pot in self.info:
            values = list(pot.values())
            if values[3] != None and values[3] != "undefined":
                result_string += f"Pot -> id = {values[0]}; capacity = {values[1]}; quantity = {values[2]};  color = {values[3]};\n"
            else:
                result_string += f"Pot -> id = {values[0]}; capacity = {values[1]}; quantity = {values[2]};\n"
        
        return result_string
    
    def __repr__(self):
        result_string = f"Node info: {str(self.info)}"+ "\n"
        if self.parent != None:
            result_string += f"Parent: {str(self.parent.info)}"+ "\n"
        else:
            result_string += f"Parent: None"+ "\n"
        result_string += f"Cost: {str(self.cost)}"+ "\n"
        result_string += f"Heuristic: {str(self.heuristic)}"+ "\n"
        result_string += f"Estimated Cost: {str(self.estimated_cost)}"+ "\n"
        result_string += f"Path Info: {str(self.path_info)}"+ "\n"
        return result_string                                   
    
    def get_path(self):
        node = self
        path = [node]
        
        while node.parent is not None:
            path.insert(0, node.parent)
            node = node.parent
        
        return path
    
    def print_path(self, print_cost = False, print_length = False):
        result_string = ""
        path = self.get_path()
        
        for node in path:
            path_info = node.path_info
            
            if path_info != None and node.parent is not None:
                values = list(path_info.values())
                # { "first_pot" : potInfo, "second_pot" : potInfo, "color" : string, "poured_liters" : number }
                result_string += f"{values[3]} liters of colored water {values[2]} were poured from the pot {values[0]} into the pot {values[1]}\n"
                result_string = node.__str__() + "\n"
                
        if print_cost:
            result_string += "Path cost: " + str(self.estimated_cost) + "\n"
        if print_length:
            result_string += "Path length: " + str(len(path)) + "\n"
            
        return result_string
    
    def contain_path(self, checked_node):
        node = self

        while node is not None:
            if node.info == checked_node:
                return True
            node = node.parent
        
        return False
    
    def has_final_state(self, final_state):
        for state_pot in final_state:
            # {"quantity" :  number, "color" : string}
            values = list(state_pot.values())
            ok = False
            for pot in self.info:
                # { "id" : number, "capacity" : number, "quantity" : number, "color" : string }
                pot_values = list(pot.values())
                if values[0] == pot_values[2] and values[1] == pot_values[3]:
                    ok = True
                    break
            if not ok:
                return ok
        return True
                
    

In [3]:
# For testing
node = Node([{ "id" : 1, "capacity" : 10, "quantity" : 8, "color" : "rosu" }], None)
print(node.__str__())
print(node.__repr__())
print(node.has_final_state([{"quantity" :  8, "color" : "red"}]))
print(node.has_final_state([{"quantity" :  8, "color" : "blue"}]))

Pot -> id = 1; capacity = 10; quantity = 8;  color = rosu;

Node info: [{'id': 1, 'capacity': 10, 'quantity': 8, 'color': 'rosu'}]
Parent: None
Cost: 0
Heuristic: 0
Estimated Cost: 0
Path Info: {}

False
False


In [4]:
class Graph:
    def __init__(self, input_filename, output_filename, output_path, timeout):
        self.timeout = timeout
        
        self.last_checked_timeout  = time.time()
        
        output_filepath = output_path + "/" + output_filename
        self.output_file = open(output_filepath, "w")
        
        self.check_timeout()
        
        try:
            reader = open(input_filename, "r")
            
            try:
                file_content = reader.read()
                
                initial_info, start_state, final_state = self.split_file(file_content)
                
                self.color_combinations, self.color_cost = self.parse_initial_info(initial_info)
                
                self.start_state = self.parse_start_state(start_state)
                
                self.final_state = self.parse_final_state(final_state)
            
            except:
                print(f"Could not parse the file! Filename:{input_filename}")
                sys.exit(0)
        except:
            print(f"Could not open the input file! Filename:{input_filename}")
            sys.exit(0)
          
    def check_timeout(self):
        current_time = time.time()
        
        time_difference = current_time - self.last_checked_timeout
        time_difference *= 1000
        
        if self.timeout < time_difference:
            print(f"Time Limit Exceeded")
            sys.exit(0)
        
    def split_file(self, file_content):
        self.check_timeout()
        
        first_split = file_content.strip().split("start_state")
        second_split = first_split[1].strip().split("final_state")
        
        initial_info = first_split[0].strip().split('\n')
        start_state = second_split[0].strip().split('\n')
        final_state = second_split[1].strip().split('\n')
        
        return initial_info, start_state, final_state
    
    
    def parse_initial_info(self, initial_info):
        self.check_timeout()
        
        color_combinations = list()
        color_cost = dict()
        
        for line in initial_info:
            content = line.split()
            if len(content) == 2:
                color_cost[content[0]] = int(content[1])
            else:
                color_combinations.append((content[0], content[1], content[2]))
        return color_combinations, color_cost
    
    
    def parse_start_state(self, start_state):
        self.check_timeout()
        
        print(start_state)
        state = []
        count = 0
        for line in start_state:
            content = line.split()
            if len(content) == 2:
                new_dict = dict()    
                new_dict = {"id" : count, "capacity" : int(content[0]), "quantity" : 0, "color" : None}
                state.append(new_dict)
            else:
                if content[2] not in self.color_cost.keys():
                    raise Exception
                new_dict = dict()    
                new_dict = {"id" : count, "capacity" : int(content[0]), "quantity" : int(content[1]), "color" : content[2]}
                state.append(new_dict)
            count += 1
        return state
    
    
    def parse_final_state(self, final_state):
        self.check_timeout()
        
        state = []
        for line in final_state:
            content = line.split()
            if content[1] not in self.color_cost.keys():
                    raise Exception
            state.append((int(content[0]),content[1]))
        return state
    
    
    def find_distinct_colors(self):
        self.check_timeout()
        
        found_colors = set()
        for (color1, color2, color3) in self.color_combinations:
            found_colors.add(color1)
            found_colors.add(color2)     
        return found_colors
    
    
    def count_combinations(self, info):
        self.check_timeout()
        
        found_colors = self.find_distinct_colors()
        number = 0
        for color in found_colors:
            for pot in info:
                values = list(pot.values())
                # "Pot -> id = {values[0]}; color = {values[3]}; quantity = {values[2]}; capacity = {values[1]}\n"
                if values[3] == color:
                    number += 1
                    break
        return number
            
    def count_final(self, info):
        self.check_timeout()
        
        number = 0
        for (color1, color2, color3) in self.color_combinations:
            for pot in info:
                values = list(pot.values())
                # "Pot -> id = {values[0]}; color = {values[3]}; quantity = {values[2]}; capacity = {values[1]}\n"
                if values[3] == color3:
                    number += 1
                    break
        return number
    
    
    def check_final(self, info):
        self.check_timeout()
        
        for (quantity, color) in self.final_state:
            count_errors = 0
            for pot in info:
#                 print(pot)
                values = list(pot.values())
                if quantity > values[1]:
                    count_errors += 1
                if count_errors == len(info):
                    return False
        return True
    
    
    def check_node(self, info):
        self.check_timeout()
        
        count_final_colors = self.count_final(info)
        count_color_combinations = self.count_combinations(info)
        check_final_state = self.check_final(info)
        
        if count_final_colors == len(self.color_combinations) and count_color_combinations == 0:
            return True
        elif count_final_colors == 0 and count_color_combinations >= 2:
            return True
        elif count_color_combinations != 0 and count_final_colors != 0:
            return True
        
        return False
    
    
    def check_color_combination(self, first_color, second_color):
        self.check_timeout()
        
        for (color1, color2, color3) in self.color_combinations:
            if ((color1 == first_color) and (color2 == second_color)) or((color1 == second_color) and (color2 == first_color)):
                return True, color3
        return False, None
                
    
    def generate_succesors(self, node, heuristic = "default"):
        self.check_timeout()
        
        succesors = list()
        count = 0
        for first_pot in node.info:
            # { "id" : number, "capacity" : number, "quantity" : number, "color" : string }
            first_pot_values = list(first_pot.values())
            
            number = 0
            for second_pot in node.info:
                if count == number:
                    continue
                
                second_pot_values = list(second_pot.values())
                new_node = copy.deepcopy(node.info)
                first_pot_copy = copy.deepcopy(first_pot_values)
                second_pot_copy = copy.deepcopy(second_pot_values)
                quantity_difference = second_pot_copy[1] - second_pot_copy[2]
                transfer_cost = 0
                color_path = dict()
                
                #  { "first_pot" : potInfo, "second_pot" : potInfo, "color" : string, "poured_liters" : number }
                color_path["first_pot"] = first_pot_copy[0] # take the id of the pots we use
                color_path["second_pot"] = second_pot_copy[0]
                
                if quantity_difference: # the second pot is not full 
                    
                    if second_pot_copy[2] != 0: # second pot is empty 
                        second_pot_copy[3] = first_pot_copy[3] 
                        color_path["color"] = first_pot_copy[3]
                        
                        if first_pot_copy[3] in self.color_cost.keys(): # this color is not undefined
                            transfer_cost += quantity_difference * self.color_cost[first_pot_copy[3]]
                        else:  # we are transfering undefined color => quantity to be moved *  1
                            transfer_cost += quantity_difference * 1
                            
                    else: # the second pot is not empty, so we are making a new color (and we need to take into consideration the color already found there)
                        if first_pot_copy[3] != None and first_pot_copy[3] == second_pot_copy[3]: # we found the same color in the pots and the color is not undefined
                            color_path["color"] = first_pot_copy[3]
                            
                            if first_pot_copy[3] in self.color_cost.keys(): # this color is not undefined
                                transfer_cost += quantity_difference * self.color_cost[first_pot_copy[3]]
                        else:
                            checker, color = self.check_color_combination(first_pot_copy[3], second_pot_copy[3])
                            if checker:
                                second_pot_copy[3] = color
                                color_path["color"] = first_pot_copy[3]
                                if first_pot_copy[3] in self.color_cost.keys(): # this color is not undefined
                                    transfer_cost += quantity_difference * self.color_cost[first_pot_copy[3]]
                                    
                            elif first_pot_copy[2] > 0: # first pot is not empty and the resulted color is undefined
                                # color cost = quantity diff *  first color cost + quantity found in second pot * second color cost
                                first_cost = second_cost = 0
                                if first_pot_copy[3] in self.color_cost.keys(): # this color is not undefined
                                    first_cost = quantity_difference * self.color_cost[first_pot_copy[3]]
                                    
                                if second_pot_copy[3] in self.color_cost.keys(): # this color is not undefined
                                    second_cost = second_pot_copy[2] * self.color_cost[second_pot_copy[3]]
                                    
                                if first_cost == 0: # first color is undefined
                                    color_path["color"] = "undefined"
                                    transfer_cost += quantity_difference * 1 
                                elif second_cost == 0: # the color is already undefined
                                    color_path["color"] = "undefined"
                                    transfer_cost += second_pot_copy[2] * 1 
                                else:
                                    color_path["color"] = first_pot_copy[3]
                                    transfer_cost += first_cost + second_cost
                                    
                                second_pot_copy[3] = None
                
                    
                    if quantity_difference > first_pot_copy[2]: # we can move more liquid than we have => the first pot will be empty
                        second_pot_copy[2] += first_pot_copy[2]
                        color_path["poured_liters"] = first_pot_copy[2]
                        first_pot_copy[2] = 0
                    else:
                        color_path["poured_liters"] = quantity_difference
                        first_pot_copy[2] -= quantity_difference
                        second_pot_copy[2] += quantity_difference
                        
                if first_pot_copy[2] == 0:
                    first_pot_copy[2] = None
                if second_pot_copy[2] == 0:
                    second_pot_copy[3] = None
                    
                c = 0
                for new_pot in new_node:
                    # "Pot -> id = {values[0]}; color = {values[3]}; quantity = {values[2]}; capacity = {values[1]}\n"
                    if c == count:
                        new_pot_values = list(new_pot.values())
                        new_pot_values[2] = copy.deepcopy(first_pot_copy[2])
                        new_pot_values[3] = copy.deepcopy(first_pot_copy[3])
                        new_dict = dict()
                        new_dict = {"id" : new_pot_values[0], "capacity" : new_pot_values[1], "quantity" : new_pot_values[2], "color" : new_pot_values[3]}

                        new_node[c] = copy.deepcopy(new_dict)
                    elif c == number:
                        new_pot_values = list(new_pot.values())
                        new_pot_values[2] = copy.deepcopy(second_pot_copy[2])
                        new_pot_values[3] = copy.deepcopy(second_pot_copy[3])
                        new_dict = dict()
                        new_dict = {"id" : new_pot_values[0], "capacity" : new_pot_values[1], "quantity" : new_pot_values[2],"color" : new_pot_values[3]}
                        new_node[c] = copy.deepcopy(new_dict)
                    c += 1
                    
                if self.check_node(new_node) and (not node.contain_path(new_node)):
                    # (self, info: list, parent,  cost = 0, heuristic = 0, path_info = {})
                    succesors.append(Node(new_node, node, node.cost + transfer_cost, self.compute_heuristic(new_node, heuristic), color_path))
                number += 1
            count += 1
        return succesors
        
        
    def find_color(self, color, info):
        for pot in info:
            values = list(pot.values())
            if values[3] == color:
                return True
        return False
        
        
    def count_color_apperances(self, color, info):
        number = 0
        for pot in info:
            values = list(pot.values())
            if values[3] == color:
                number += 1
        return number
        
        
    def compute_heuristic(self, node_info, heuristic = "default"):
        self.check_timeout()
        
        # default heuristic
        if heuristic == "default":
            if self.check_final(node_info):
                return 1
            return 0
        elif heuristic == "admissible 1":
            
            # compunte the number of transfers required to get the colors specified in final states 
            # the heuristic is the minimum number
            # for pots that are already in final state, the heuristic equals to 0
            
            heuristics = list()
            
            for (quantity, color) in self.final_state:
                current_heuristic = 0
                
                if self.find_color(color, node_info) == False:
                    for (color1, color2, color3) in self.color_combinations:
                        cost_color1 = self.cost[color1]
                        cost_color2 = self.cost[color2]
                        
                        color1_apperances = self.count_color_apperances(color1, node_info)
                        color2_apperances = self.count_color_apperances(color2, node_info)
                        
                        color1_total = cost_color1 * color1_apperances
                        color2_total = cost_color2 * color2_apperances
                        
                        current_heuristic = min(color1_total, color2_total)
                
                heuristics.append(current_heuristic)
            
            self.check_timeout()
            
            return min(heuristics)
        
        elif heuristic == "admissible 2":
            
            # for pots that are already in final state, the heuristic equals to 0
            # we compute the minimum cost between the colors that can be mixed in order to get a final state color
            
            heuristics = list()
                
            for (quantity, color) in self.final_state:
                current_heuristic = 0
                
                if self.find_color(color, node_info) == False:
                    for (color1, color2, color3) in self.color_combinations:
                        cost_color1 = self.cost[color1]
                        cost_color2 = self.cost[color2]
                        
                        current_heuristic = min(cost_color1, cost_color2)
                
                heuristics.append(current_heuristic)
                
            self.check_timeout()
                
            return min(heuristics)
            
        elif heuristic == "inadmissible":
            
            # an inadmissible heuristic can be created by changing the second admissible heuristic
            # instead of taking the minimum between the costs, we can take the maximum value 
            # and to be sure it is inadmissible, we take teh square value
            
            heuristics = list()
                
            for (quantity, color) in self.final_state:
                current_heuristic = 0
                
                if self.find_color(color, node_info) == False:
                    for (color1, color2, color3) in self.color_combinations:
                        cost_color1 = self.cost[color1]
                        cost_color2 = self.cost[color2]
                        
                        current_heuristic = max(cost_color1, cost_color2)
                        current_heuristic *= current_heuristic
                
                heuristics.append(current_heuristic)
                
            self.check_timeout()
                
            return max(heuristics)                                                                                          
        

In [5]:
def a_star(graph, number_of_solutions = 1, heuristic = "default"):
    
    unexpanded = list()
    node = Node(graph.start_state, None, 0, graph.compute_heuristic(graph.start, heuristic),{})
    unexpanded.append(node)

    expanded = list()
    
    
    

In [6]:
input_text = """
rosu galben portocaliu
rosu 2
galben 3
portocaliu 5
start_state
3 2 rosu
2 1 galben
4 2 portocaliu
final_state
2 rosu
1 galben
2 portocaliu
"""

with open("input.txt", "w+") as fin:
  fin.write(input_text)


In [7]:
graph = Graph("./input.txt", "./file.txt", "./", 5000)

['3 2 rosu', '2 1 galben', '4 2 portocaliu']


In [8]:
graph.start_state

[{'id': 0, 'capacity': 3, 'quantity': 2, 'color': 'rosu'},
 {'id': 1, 'capacity': 2, 'quantity': 1, 'color': 'galben'},
 {'id': 2, 'capacity': 4, 'quantity': 2, 'color': 'portocaliu'}]

In [9]:
node1 = Node(graph.start_state, None, 0, 0, [])

In [10]:
node1.info

[{'id': 0, 'capacity': 3, 'quantity': 2, 'color': 'rosu'},
 {'id': 1, 'capacity': 2, 'quantity': 1, 'color': 'galben'},
 {'id': 2, 'capacity': 4, 'quantity': 2, 'color': 'portocaliu'}]

In [11]:
graph.color_combinations

[('rosu', 'galben', 'portocaliu')]

In [12]:
graph.final_state

[(2, 'rosu'), (1, 'galben'), (2, 'portocaliu')]

In [13]:
graph.generate_succesors(node1)

[Node info: [{'id': 0, 'capacity': 3, 'quantity': 3, 'color': 'galben'}, {'id': 1, 'capacity': 2, 'quantity': None, 'color': 'galben'}, {'id': 2, 'capacity': 4, 'quantity': 2, 'color': 'portocaliu'}]
 Parent: [{'id': 0, 'capacity': 3, 'quantity': 2, 'color': 'rosu'}, {'id': 1, 'capacity': 2, 'quantity': 1, 'color': 'galben'}, {'id': 2, 'capacity': 4, 'quantity': 2, 'color': 'portocaliu'}]
 Cost: 3
 Heuristic: 1
 Estimated Cost: 4
 Path Info: {'first_pot': 1, 'second_pot': 0, 'color': 'galben', 'poured_liters': 1},
 Node info: [{'id': 0, 'capacity': 3, 'quantity': 3, 'color': 'portocaliu'}, {'id': 1, 'capacity': 2, 'quantity': 1, 'color': 'galben'}, {'id': 2, 'capacity': 4, 'quantity': 1, 'color': 'portocaliu'}]
 Parent: [{'id': 0, 'capacity': 3, 'quantity': 2, 'color': 'rosu'}, {'id': 1, 'capacity': 2, 'quantity': 1, 'color': 'galben'}, {'id': 2, 'capacity': 4, 'quantity': 2, 'color': 'portocaliu'}]
 Cost: 5
 Heuristic: 1
 Estimated Cost: 6
 Path Info: {'first_pot': 2, 'second_pot': 0,

In [14]:
# .--.      .--.   ____   ,---------.    .-''-.  .-------.            .-------.     ,-----.  ,---------.   .-'''-.  
# |  |_     |  | .'  __ `.\          \ .'_ _   \ |  _ _   \           \  _(`)_ \  .'  .-,  '.\          \ / _     \ 
# | _( )_   |  |/   '  \  \`--.  ,---'/ ( ` )   '| ( ' )  |           | (_ o._)| / ,-.|  \ _ \`--.  ,---'(`' )/`--' 
# |(_ o _)  |  ||___|  /  |   |   \  . (_ o _)  ||(_ o _) /           |  (_,_) /;  \  '_ /  | :  |   \  (_ o _).    
# | (_,_) \ |  |   _.-`   |   :_ _:  |  (_,_)___|| (_,_).' __         |   '-.-' |  _`,/ \ _/  |  :_ _:   (_,_). '.  
# |  |/    \|  |.'   _    |   (_I_)  '  \   .---.|  |\ \  |  |        |   |     : (  '\_/ \   ;  (_I_)  .---.  \  : 
# |  '  /\  `  ||  _( )_  |  (_(=)_)  \  `-'    /|  | \ `'   /        |   |      \ `"/  \  ) /  (_(=)_) \    `-'  | 
# |    /  \    |\ (_ o _) /   (_I_)    \       / |  |  \    /         /   )       '. \_/``".'    (_I_)   \       /  
# `---'    `---` '.(_,_).'    '---'     `'-..-'  ''-'   `'-'          `---'         '-----'      '---'    `-...-'   
                                                                                                                  